In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
csv = pd.read_csv('./Data/BTC_2019_2023_30m.csv')

In [3]:
csv = csv.rename(columns={"Unnamed: 0	":'index'})

In [4]:
csv.columns.values[0] = "user_id"

In [7]:
csv.head
df = csv
df = df.assign(signals = 0)
df = df.assign(trade_type = 'NaN')

In [9]:

# Calculate EMA
df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()
df['ema_40'] = df['close'].ewm(span=40, adjust=False).mean()

def generate_signals(df):
    signals = []
    for i in range(len(df)):
        if i == 0:
            signals.append(0)
        elif df['ema_20'][i] > df['ema_40'][i] and df['ema_20'][i-1] <= df['ema_40'][i-1]:
            signals.append(1)
        elif df['ema_20'][i] < df['ema_40'][i] and df['ema_20'][i-1] >= df['ema_40'][i-1]:
            signals.append(-1)
        else:
            signals.append(0)
    return signals

df['signals'] = generate_signals(df)

def determine_trade_type(row):
    if row['signals'] == 1 and (pd.isna(row['trade_type']) or row['trade_type'] == 'close'):
        return 'long'
    elif row['signals'] == -1 and (pd.isna(row['trade_type']) or row['trade_type'] == 'close'):
        return 'short'
    elif row['signals'] == -1 and row['trade_type'] == 'long':
        return 'close'
    elif row['signals'] == 1 and row['trade_type'] == 'short':
        return 'close'
    elif row['signals'] == -2:
        return 'short'
    elif row['signals'] == 2:
        return 'long'
    else:
        return row['trade_type']

df['trade_type'] = df.apply(determine_trade_type, axis=1)
df['trade_type'].fillna('hold', inplace=True)


         user_id             datetime     open     high      low    close  \
0              0  2019-09-08 17:57:00  10000.0  10000.0  10000.0  10000.0   
1              1  2019-09-08 17:58:00  10000.0  10000.0  10000.0  10000.0   
2              2  2019-09-08 17:59:00  10000.0  10000.0  10000.0  10000.0   
3              3  2019-09-08 18:00:00  10000.0  10000.0  10000.0  10000.0   
4              4  2019-09-08 18:01:00  10000.0  10000.0  10000.0  10000.0   
...          ...                  ...      ...      ...      ...      ...   
2268358  2268358  2023-12-31 23:56:00  42258.7  42274.4  42250.5  42274.4   
2268359  2268359  2023-12-31 23:57:00  42274.3  42315.6  42274.3  42310.9   
2268360  2268360  2023-12-31 23:58:00  42310.9  42314.5  42307.7  42312.6   
2268361  2268361  2023-12-31 23:59:00  42312.7  42314.0  42293.7  42314.0   
2268362  2268362  2024-01-01 00:00:00  42314.0  42335.8  42289.6  42331.9   

          volume  signals trade_type        ema_20        ema_40  
0       

C:\Users\entro\AppData\Local\Temp\ipykernel_11840\3108071952.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['trade_type'].fillna('hold', inplace=True)


In [10]:
df.to_csv ('./content/ema_result.csv', index=False)